<a href="https://colab.research.google.com/github/blanco-herrero/TriPol-project/blob/main/David_historical_tweets_apiv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Full search archive for Academic Track
Adapted from: https://github.com/rik-r/twitter_full_archive/blob/master/historical_tweets_apiv2.ipynb

This Jupyter notebook allows you to use the full archive search endpoint for Twitter's API v2.
You must be approved for the Academic Research track to use this script.

The last cell of the notebook allows you to specify the parameters for your search. You will need a bearer token to authenticate.

NOTE: The endpoint allows you to make only 1 request per second and 300 requests per 15 minutes. Accordingly, if your query returns more than 150k tweets, this script will sleep for 10 minutes before resuming.

In [ ]:
import requests
import pandas as pd
import json
import time

In [ ]:
def search_endpoint_connect(bearer_token, query, st, et, next_token):
    
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    query_params = {
                    'query': query,
                    'start_time': st,
                    'end_time': et,
                    'max_results': 10,
                    'expansions': 'author_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,created_at,geo,lang,public_metrics,in_reply_to_user_id',
                    'user.fields': 'username,name,public_metrics',
                    'place.fields': 'id,full_name,name,place_type,country,country_code',
                   }
    
    if (next_token is not None):
        url = "https://api.twitter.com/2/tweets/search/all?next_token={}".format(next_token)
    else:
        url = "https://api.twitter.com/2/tweets/search/all"
    
    response = requests.request("GET", url, params=query_params, headers=headers)
    
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    
    return response.json()

In [ ]:
def main(bearer_token, n, fn, sq, st, et):
    
    rl_count = 0
    count = 0
    flag = True
    first = True


    while flag:
        
        if rl_count==300:
            time.sleep(600)
            print('Rate limit cooldown 10 mins.')

        if count >= n and n!=4000:
            break
        if not first:
            json_response = search_endpoint_connect(bearer_token, sq, st, et, next_token)
        else:
            json_response = search_endpoint_connect(bearer_token, sq, st, et, next_token=None)
        
        result_count = json_response['meta']['result_count']
        if 'next_token' in json_response['meta']:
            next_token = json_response['meta']['next_token']
        
            if result_count is not None and result_count > 0 and next_token is not None:
                
                df = pd.json_normalize(json_response['data'])
                df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
                df = df.reindex(columns=['id', 'author_id', 'created_at', 'text', 'lang', 
                                    'public_metrics.retweet_count', 'public_metrics.reply_count', 'public_metrics.like_count', 'public_metrics.quote_count', 
                                    'in_reply_to_user_id', 'geo.place_id', 'geo.coordinates.type', 'geo.coordinates.coordinates'])
            
            
            
                #df2 = pd.json_normalize(json_response['includes']['users'])
                #df2.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
                #df2 = df2.reindex(columns=['id','username','name',
                #                           "public_metrics.followers_count", "public_metrics.following_count", 
                #                           "public_metrics.listed_count", "public_metrics.tweet_count"])
                
                #df4 = pd.json_normalize(json_response['includes']['places'])
                #df4.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
                #df4 = df4.reindex(columns=['id','full_name','name','place_type','country','country_code'])
                
                
                #df3 = df.merge(df2, left_on='author_id', right_on='id')
                
                #df5 = df4.merge(df3, left_on='id', right_on='geo.place_id')
                
                if not first:
                    df.to_csv('%s.csv'%fn, mode='a', encoding='utf-8', sep= ";", index=False, header=None)
                else:
                    df.to_csv('%s.csv'%fn, encoding='utf-8', sep= ";", index=False)

                time.sleep(1)
                count += result_count
                print('Tweets downloaded: '+str(count))
        else:
            flag = False
        rl_count += 1
        first = False

In [ ]:
#Enter your bearer token
bearer_token = ''

#Set number of tweets to be downloaded. Enter 0 for no limits
no_of_tweets = 10000

#Specify the name of the output csv file. Do not include .csv
file_name = 'downloaded_tweets'

#Enter your search query. Refer to https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query
search_query = '(@sanchezcastejon) lang:ES place_country:ES -is:retweet'

#Set the beginning date and time in YYYY-MM-DDTHH:MM:SSZ format
start_time = '2022-01-01T00:00:00Z'

#Set the ending date and time in YYYY-MM-DDTHH:MM:SSZ format
end_time = '2022-11-10T00:00:00.00Z'

main(bearer_token, no_of_tweets, file_name, search_query, start_time, end_time)

Tweets downloaded: 10
Tweets downloaded: 20
Tweets downloaded: 30
Tweets downloaded: 40
Tweets downloaded: 50
Tweets downloaded: 60
Tweets downloaded: 70
Tweets downloaded: 80
Tweets downloaded: 89
Tweets downloaded: 99
Tweets downloaded: 109
Tweets downloaded: 119
Tweets downloaded: 129
Tweets downloaded: 139
Tweets downloaded: 149
Tweets downloaded: 159
Tweets downloaded: 169
Tweets downloaded: 179
Tweets downloaded: 189
Tweets downloaded: 199
Tweets downloaded: 209
Tweets downloaded: 219
Tweets downloaded: 229
Tweets downloaded: 239
Tweets downloaded: 249
Tweets downloaded: 259
Tweets downloaded: 269
Tweets downloaded: 279
Tweets downloaded: 289
Tweets downloaded: 299
Tweets downloaded: 309
Tweets downloaded: 319
Tweets downloaded: 329
Tweets downloaded: 339
Tweets downloaded: 349
Tweets downloaded: 359
Tweets downloaded: 369
Tweets downloaded: 379
Tweets downloaded: 389
Tweets downloaded: 399
Tweets downloaded: 409
Tweets downloaded: 419
Tweets downloaded: 429
Tweets downloaded: 43

In [ ]:
from google.colab import files
files.download('downloaded_tweets.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!cat downloaded_tweets.csv

id;author_id;created_at;text;lang;public_metrics.retweet_count;public_metrics.reply_count;public_metrics.like_count;public_metrics.quote_count;in_reply_to_user_id;geo.place_id;geo.coordinates.type;geo.coordinates.coordinates
1590131951154757632;2288743842;2022-11-08T23:59:35.000Z;karma is definitely real and his no expectation date. but for people to constantly torment Erica Mena for the shit she did IN HER 20’s is crazy. Some people live and learn. Them kids didn’t ask to be here!;en;0;1;2;1;;;;
1590131944377176067;1503719349626019844;2022-11-08T23:59:33.000Z;Yo en junio ya asumí que Mena no va a seguir en Racing el próximo año. Les aviso por si alguno tiene esperanzas.;es;1;9;57;0;;;;
1590131778869956608;1587299262676848642;2022-11-08T23:58:53.000Z;@Itsmelanie713 @Mena_Mahan this person @Mena_Mahan is a doxxer and dangerous;en;0;0;0;0;1501283698611683331;;;
1590131649085599752;1436534596833775616;2022-11-08T23:58:23.000Z;@SailiClara @addi_mena @Robert_DeCoito @dkramelo_ Jjaajjaja es 